# Prepare Files for qpAdm

In [1]:
import numpy as np
import os  # For Saving to Folder
import pandas as pd
import matplotlib.pyplot as plt

import socket
import os as os
import sys as sys
import multiprocessing as mp
import itertools as it
from time import time

# For Arial Font
from matplotlib import rcParams
rcParams['font.family'] = 'sans-serif'   # Set the defaul
# Make sure to have the font installed (it is on cluster for Harald)
rcParams['font.sans-serif'] = ['Arial']

socket_name = socket.gethostname()
print(socket_name)

if socket_name.startswith("compute-"):
    print("HSM Computational partition detected.")
    path = "/n/groups/reich/hringbauer/git/punic_aDNA/"  # The Path on Midway Cluster
else:
    raise RuntimeWarning("Not compatible machine. Check!!")

os.chdir(path)  # Set the right Path (in line with Atom default)
# Show the current working directory. Should be HAPSBURG/Notebooks/ParallelRuns
print(os.getcwd())
print(f"CPU Count: {mp.cpu_count()}")
print(sys.version)

compute-e-16-235.o2.rc.hms.harvard.edu
HSM Computational partition detected.
/n/groups/reich/hringbauer/git/punic_aDNA
CPU Count: 28
3.7.4 (default, Sep 11 2019, 11:24:51) 
[GCC 6.2.0]


# Key Definitions

In [2]:
base_path = "/n/groups/reich/DAVID/V42/V42.3/v42.3"
ind_path = base_path + ".ind"

In [6]:
#a13_new = ["Sweden_HG_Motala", "Russia_Ust_Ishim.DG", "Russia_Kostenki14.SG",
#          "Goyet_Neanderthal.SG", "Czech_Vestonice16", "Russia_MA1_HG.SG",  "Iberia_ElMiron",
#           "Italy_North_HG_Villabruna", "Russia_HG_Karelia", "Israel_Natufian", "Iran_N_GanjDareh"]
#
#distal = ["Anatolia_N", "Iberia_HG", 
#          "Morocco_Iberomaurusian", 
#          "Russia_EBA_Yamnaya_Samara"]

#print("M13: %i popoulations" % len(m13))
#print("Anc_Euro: %i populations" % len(a13_new))

# Save Population File (what to pull)

In [30]:
def return_pops(df, string, col="clst", 
                output=False):
    """Return list of clusters that contain string."""
    df1 = df[df[col].str.contains(string)]
    if output:
        print(df1[col].value_counts())
    clsts = list(set(df1[col].values))

    return clsts

In [29]:
df_ind = pd.read_csv(ind_path, delim_whitespace=True, header=None)
df_ind.columns=["iid", "sex", "clst"]
print(f"Loaded {len(df_ind)} Individuals")

Loaded 16667 Individuals


In [ ]:
##############################################
### Testing for single Populations Populations
return_pops(df_ind, string="Sardinia", 
            output=True);

In [118]:
pops = ["Algeria", "Morocco", "Punic", "Phoenician", 
        "Canaanite", "Ashkelon", "Greek_Sicily", "Sicily_IA",
        "Israel_IA", "Israel_EIA", "Israel_Persian", "Gibraltar",
        "Iberia_North_BA_Africa_all", "Iberia_BellBeaker_o", "Iberia_Greek",
        "Iberia_Hellenistic"]

exclude_strings = ["_lc", "contam"]

In [131]:
clsts = [return_pops(df_ind, string=pop, 
                     output=False) for pop in pops]

clsts = [inner for ls in clsts for inner in ls]
clsts = list(set(clsts)) # Filter to unique Elements
print(f"Loaded {len(clsts)} Populations")

### Exclude Strings
for ex in exclude_strings:
    clsts = [c for c in clsts if ex not in c]
print(f"After Exclusion {len(clsts)}")

Loaded 66 Populations
After Exclusion 50


# Save List of populations to keep

In [132]:
#keep = np.array(["Anatolia_N", "Iberia_HG"])
keep = np.array(clsts)
path_keep = "./explore_ntbk/parfiles/keep_pops"
np.savetxt(path_keep, keep, fmt="%s")

### Run convertf (with population list to keep)

In [133]:
def run_convertf(path_convertf = "./o2bin/convertf", parfile = "./explore_ntbk/parfiles/convertf.keep.par"):
    """Runs the Downsampling"""
    ! $path_convertf -p $parfile

In [134]:
%%time
run_convertf(path_convertf = "./o2bin/convertf", 
             parfile = "./explore_ntbk/parfiles/convertf.keep.par")

parameter file: ./explore_ntbk/parfiles/convertf.keep.par
BASE: /n/groups/reich/hringbauer
DIR: explore_ntbk/v42/V42.3/v42.3
OUT: explore_ntbk/eigenstrat/punic0.v43
genotypename: /n/groups/reich/hringbauer/explore_ntbk/v42/V42.3/v42.3.geno
snpname: /n/groups/reich/hringbauer/explore_ntbk/v42/V42.3/v42.3.snp
indivname: /n/groups/reich/hringbauer/explore_ntbk/v42/V42.3/v42.3.ind
genooutfilename: /n/groups/reich/hringbauer/explore_ntbk/eigenstrat/punic0.v43.geno
snpoutfilename: /n/groups/reich/hringbauer/explore_ntbk/eigenstrat/punic0.v43.snp
indoutfilename: /n/groups/reich/hringbauer/explore_ntbk/eigenstrat/punic0.v43.ind
outputformat: PACKEDANCESTRYMAP
hashcheck: YES
poplistname: /n/groups/reich/hringbauer/explore_ntbk/parfiles/keep_pops
## ./o2bin/convertf version: 5711
read 1073741824 bytes
read 2147483648 bytes
read 3221225472 bytes
read 4294967296 bytes
read 5137965171 bytes
packed geno read OK
end of inpack
before compress: snps: 1233013 indivs: 16667
after compress: snps: 1233013 

## Merge in Lazaridis Ancients with mergeit (with population list to keep)

In [8]:
bin_merge_it = "/n/groups/reich/hringbauer/o2bin/mergeit"
parfile_path = "./parfiles/parMergeAddAnc"

In [10]:
%%time
! $bin_merge_it -p $parfile_path

parameter file: ./parfiles/parMergeAddAnc
BASE: /n/groups/reich/hringbauer/git/punic_aDNA
S1: eigenstrat/punic0.v43
S2: eigenstrat/additional/MinMyc
OUT: eigenstrat/combined/punic0.v43
geno1: /n/groups/reich/hringbauer/git/punic_aDNA/eigenstrat/punic0.v43.geno
snp1: /n/groups/reich/hringbauer/git/punic_aDNA/eigenstrat/punic0.v43.snp
ind1: /n/groups/reich/hringbauer/git/punic_aDNA/eigenstrat/punic0.v43.ind
geno2: /n/groups/reich/hringbauer/git/punic_aDNA/eigenstrat/additional/MinMyc.geno
snp2: /n/groups/reich/hringbauer/git/punic_aDNA/eigenstrat/additional/MinMyc.snp
ind2: /n/groups/reich/hringbauer/git/punic_aDNA/eigenstrat/additional/MinMyc.ind
genooutfilename: /n/groups/reich/hringbauer/git/punic_aDNA/eigenstrat/combined/punic0.v43.geno
snpoutfilename: /n/groups/reich/hringbauer/git/punic_aDNA/eigenstrat/combined/punic0.v43.snp
indoutfilename: /n/groups/reich/hringbauer/git/punic_aDNA/eigenstrat/combined/punic0.v43.ind
docheck: YES
hashcheck: NO
allowdups: YES
packed geno read OK
end

# Prepare Individual File [Stand Alone]

In [12]:
ind_merged="./eigenstrat/combined/punic0.v43.ind"          # What .ind to load
df_ind = pd.read_csv(ind_merged, delim_whitespace=True, header=None)
df_ind.columns=["iid", "sex", "clst"]
print(f"Loaded {len(df_ind)} Individuals")

Loaded 527 Individuals


In [25]:
def overwrite_ind_df(df, string, col="clst", 
                     output=False, overwrite="", iids=False):
    """Overwrite Individual Dataframe where col
    contains string. Return modified dataframe (Copy)
    where overwrite is the new Cluster ID
    iids: Overwrite with IIDs if given!"""
    idx = df[col].str.contains(string)
    
    if np.sum(idx)==0:
        if output: 
            print("No Indivdiuals found")
        return
    
    if output:
        print(f"Found {np.sum(idx)} Matches")
        print(df[idx][col].value_counts())
    
    ### Actually  overwrite the Column
    if len(overwrite)>0:
        df.loc[idx, col] = overwrite
        if output: 
            print(f"{np.sum(idx)} Overwritten!")
            
    if iids:
        df.loc[idx, col] = df.loc[idx, "iid"] 

### Test around [don't run]

In [24]:
overwrite_ind_df(df_ind, "Iberia_BellBeaker_o", output=True, overwrite="")

Found 4 Matches
Iberia_BellBeaker_o              2
Iberia_BellBeaker_o2             1
Iberia_BellBeaker_o_published    1
Name: clst, dtype: int64


### Actually overwrite

In [26]:
### Overwrite Group Names
overwrite_ind_df(df_ind, "Morocco_LN", overwrite="Morocco_LN")
overwrite_ind_df(df_ind, "Morocco_EN", overwrite="Morocco_EN")
overwrite_ind_df(df_ind, "Morocco_Iberomaurusian", overwrite="Morocco_Iberomaurusian")

In [27]:
### Overwrite Individual IIDs
overwrite_ind_df(df_ind, "Algeria_N", 
                 iids=True, output=True)
overwrite_ind_df(df_ind, "Punic", 
                 iids=True, output=True)
overwrite_ind_df(df_ind, "Italy_Sicily_Phoenician", 
                 iids=True, output=True);
overwrite_ind_df(df_ind, "Morocco_LN", 
                 iids=True, output=True)
overwrite_ind_df(df_ind, "Punic_oAfrican", 
                 iids=True, output=True)
overwrite_ind_df(df_ind, "Iberia_North_BA_Africa_all", 
                 iids=True, output=True)
overwrite_ind_df(df_ind, "Iberia_BellBeaker_o", 
                 iids=True, output=True)

No Indivdiuals found
No Indivdiuals found
No Indivdiuals found
No Indivdiuals found
No Indivdiuals found
No Indivdiuals found
Found 4 Matches
Iberia_BellBeaker_o              2
Iberia_BellBeaker_o2             1
Iberia_BellBeaker_o_published    1
Name: clst, dtype: int64


### Save to Drive here
mod.ind is standard (Morocco_LN not split up)  
mod1.ind is with Morocc_LN split up  

In [28]:
### Save
ind_modified="./eigenstrat/combined/punic0.v43_mod1.ind"    # Where to save the modified version to
df_ind.to_csv(ind_modified, sep=" ", index=None, header=False)
print(f"Saved {len(df_ind)} Individuals to {ind_modified}")

Saved 527 Individuals to ./eigenstrat/combined/punic0.v43_mod1.ind


# Area 51

In [95]:
ind_merged="./eigenstrat/combined/punic0.v43.ind"          # What .ind to load
df_ind = pd.read_csv(ind_merged, delim_whitespace=True, header=None)
df_ind.columns=["iid", "sex", "clst"]
print(f"Loaded {len(df_ind)} Individuals")

Loaded 527 Individuals


In [97]:
overwrite_ind_df(df_ind, "Sardinia", output=True)

Found 9 Matches
Italy_Sardinia_IA_Punic    9
Name: clst, dtype: int64


In [98]:
df_ind[df_ind["clst"].str.contains("Sardinia")]

,iid,sex,clst
165,MSR002,M,Italy_Sardinia_IA_Punic
166,MSR003,F,Italy_Sardinia_IA_Punic
167,ORC002,M,Italy_Sardinia_IA_Punic
168,VIL004,F,Italy_Sardinia_IA_Punic
169,VIL006,F,Italy_Sardinia_IA_Punic
170,VIL007,M,Italy_Sardinia_IA_Punic
171,VIL009,F,Italy_Sardinia_IA_Punic
172,VIL010,F,Italy_Sardinia_IA_Punic
173,VIL011,M,Italy_Sardinia_IA_Punic
